In [1]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split, cross_val_score
# from tpot.config.classifier_nn import classifier_config_nn

from sklearn.pipeline import make_pipeline
# from tpot.config import classifier_config_dict_light
from tpot.config import classifier_config_dict
from sklearn.neighbors import KNeighborsClassifier


import pandas as pd
import numpy as np
import os
import glob

import time

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
pd.options.mode.chained_assignment = None

Using TensorFlow backend.


In [2]:
# personal_config = classifier_config_dict_light
personal_config = classifier_config_dict
personal_config['tpot.builtins.SimpleAutoencoder'] = {
    'encoding_dim': [10, 50, 100, 500],
    'activation': ['relu'],
    'optimizer': ['adadelta', "SGD", "Adam", "Adamax", "Nadam"],
    'loss':['binary_crossentropy', 'hinge', 'mean_squared_error', 'mean_absolute_error'],
    'epochs':[50, 100],
    'batch_size':[10, 15]
}

# personal_config['tpot.builtins.SimpleAutoencoder'] = {
#     'regularizer' :['regularizers.l1(10e-5)', 'regularizers.l2(10e-5)', 'regularizers.l1_l2(10e-5, 10e-5)'],
#     'encoding_dim': [40],
#     'activation': ['relu'],
#     'optimizer': ['adadelta', "SGD"],
#     'loss':['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error',
#        'squared_hinge', 'hinge', 'logcosh', 'binary_crossentropy', 'kullback_leibler_divergence', 'cosine_proximity', 'poisson'],
#     'epochs':[100],
#     'batch_size':[200]
# }

In [3]:
#new data
new_data = pd.read_csv("processed_aml_data.csv")
X = new_data.iloc[:, 1:5051]
y = new_data.iloc[:, 5051]

In [4]:
#convert X and y to numpy
X = X.values
y = y.values



In [5]:
#split data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(X, y, train_size = 0.8, test_size = 0.20, random_state = 42)
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_val = x_val.reshape((len(x_val)), np.prod(x_val.shape[1:]))

In [6]:
x_train.shape

(42, 5050)

In [7]:
y_train.shape

(42,)

In [14]:
from tpot.builtins import SimpleAutoencoder

In [15]:
sae = SimpleAutoencoder(encoding_dim=10, 
                        activation='relu', 
                        optimizer='adadelta', 
                        loss='binary_crossentropy', 
                        epochs=50, batch_size=10)

In [16]:
sae.fit(x_train, y_train)

SimpleAutoencoder(activation='relu', batch_size=10, encoding_dim=10,
         epochs=50, loss='binary_crossentropy', optimizer='adadelta',
         random_state=42)

In [17]:
tpot = TPOTClassifier(generations=50, config_dict=personal_config,
                        population_size=100, verbosity=3,
                        template = 'SimpleAutoencoder-RandomForestClassifier')

In [18]:
start = time.time()
print("time")
end = time.time()
print(end-start)

time
0.000997304916381836


In [19]:
start = time.time()
tpot.fit(x_train, y_train)
end = time.time()
print(end - start)

31 operators have been imported by TPOT.


Skipped pipeline #55 due to time out. Continuing to the next pipeline.
Skipped pipeline #88 due to time out. Continuing to the next pipeline.
Skipped pipeline #93 due to time out. Continuing to the next pipeline.
Skipped pipeline #95 due to time out. Continuing to the next pipeline.
Skipped pipeline #99 due to time out. Continuing to the next pipeline.
Skipped pipeline #105 due to time out. Continuing to the next pipeline.
Skipped pipeline #109 due to time out. Continuing to the next pipeline.
Skipped pipeline #111 due to time out. Continuing to the next pipeline.
Skipped pipeline #113 due to time out. Continuing to the next pipeline.
Skipped pipeline #115 due to time out. Continuing to the next pipeline.
Skipped pipeline #117 due to time out. Continuing to the next pipeline.
Skipped pipeline #119 due to time out. Continuing to the next pipeline.
Skipped pipeline #121 due to time out. Continuing to the next pipeline.
Skipped pipeline #123 due to time out. Continuing to the next pipelin

In [12]:
tpot.fitted_pipeline_

Pipeline(memory=None,
     steps=[('simpleautoencoder', SimpleAutoencoder(activation='relu', batch_size=15, encoding_dim=50,
         epochs=10, loss='binary_crossentropy', optimizer='adadelta',
         random_state=42)), ('randomforestclassifier', RandomForestClassifier(bootstrap=False, class_weight=None,
            criter...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [13]:
tpot.score(x_train, y_train)

0.8571428571428571

In [14]:
tpot.score(x_val, y_val)

0.6363636363636364